# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 12:24PM,248060,12,HH-37357,Hush Hush,Released
1,Oct 6 2022 12:24PM,248060,12,HH-37358,Hush Hush,Released
2,Oct 6 2022 12:24PM,248060,12,HH-37359,Hush Hush,Released
3,Oct 6 2022 12:23PM,248061,16,8618064,Sartorius Bioprocess Solutions,Released
4,Oct 6 2022 12:11PM,248059,10,MSP214529,"Methapharm, Inc.",Released
5,Oct 6 2022 12:11PM,248059,10,MSP214481,"Methapharm, Inc.",Released
6,Oct 6 2022 12:11PM,248059,10,MSP214575,"Methapharm, Inc.",Released
7,Oct 6 2022 12:11PM,248059,10,MSP214616,"Methapharm, Inc.",Released
8,Oct 6 2022 12:11PM,248059,10,MSP214627,"Methapharm, Inc.",Released
9,Oct 6 2022 12:11PM,248059,10,MSP214661,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,248055,Released,1
23,248057,Released,1
24,248059,Released,13
25,248060,Released,3
26,248061,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248055,NaN,NaN,1.0
248057,NaN,NaN,1.0
248059,NaN,NaN,13.0
248060,NaN,NaN,3.0
248061,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247922,0.0,36.0,8.0
247943,0.0,1.0,0.0
247985,0.0,0.0,3.0
247991,0.0,0.0,1.0
248000,0.0,6.0,14.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247922,0,36,8
247943,0,1,0
247985,0,0,3
247991,0,0,1
248000,0,6,14


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247922,0,36,8
1,247943,0,1,0
2,247985,0,0,3
3,247991,0,0,1
4,248000,0,6,14


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247922,,36,8
1,247943,,1,
2,247985,,,3
3,247991,,,1
4,248000,,6,14


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 12:24PM,248060,12,Hush Hush
3,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions
4,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc."
17,Oct 6 2022 12:08PM,248050,10,"Methapharm, Inc."
32,Oct 6 2022 11:49AM,248057,19,"GCH Granules USA, Inc."
33,Oct 6 2022 11:43AM,248055,10,"Citieffe, Inc."
34,Oct 6 2022 11:36AM,248052,16,"SHL Pharma, LLC"
35,Oct 6 2022 11:35AM,248054,10,"ClearSpec, LLC"
36,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc."
37,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 12:24PM,248060,12,Hush Hush,,,3
1,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,,,1
2,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",,,13
3,Oct 6 2022 12:08PM,248050,10,"Methapharm, Inc.",,,15
4,Oct 6 2022 11:49AM,248057,19,"GCH Granules USA, Inc.",,,1
5,Oct 6 2022 11:43AM,248055,10,"Citieffe, Inc.",,,1
6,Oct 6 2022 11:36AM,248052,16,"SHL Pharma, LLC",,,1
7,Oct 6 2022 11:35AM,248054,10,"ClearSpec, LLC",,,1
8,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",,,1
9,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,,13,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 12:24PM,248060,12,Hush Hush,3,,
1,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,1,,
2,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",13,,
3,Oct 6 2022 12:08PM,248050,10,"Methapharm, Inc.",15,,
4,Oct 6 2022 11:49AM,248057,19,"GCH Granules USA, Inc.",1,,
5,Oct 6 2022 11:43AM,248055,10,"Citieffe, Inc.",1,,
6,Oct 6 2022 11:36AM,248052,16,"SHL Pharma, LLC",1,,
7,Oct 6 2022 11:35AM,248054,10,"ClearSpec, LLC",1,,
8,Oct 6 2022 11:14AM,248049,19,"GUSA Granules USA, Inc.",1,,
9,Oct 6 2022 11:12AM,248042,10,Eywa Pharma Inc.,8,13,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 12:24PM,248060,12,Hush Hush,3,,
1,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,1,,
2,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",13,,
3,Oct 6 2022 12:08PM,248050,10,"Methapharm, Inc.",15,,
4,Oct 6 2022 11:49AM,248057,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 12:24PM,248060,12,Hush Hush,3.0,NaN,NaN
1,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
2,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",13.0,NaN,NaN
3,Oct 6 2022 12:08PM,248050,10,"Methapharm, Inc.",15.0,NaN,NaN
4,Oct 6 2022 11:49AM,248057,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 12:24PM,248060,12,Hush Hush,3.0,0.0,0.0
1,Oct 6 2022 12:23PM,248061,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
2,Oct 6 2022 12:11PM,248059,10,"Methapharm, Inc.",13.0,0.0,0.0
3,Oct 6 2022 12:08PM,248050,10,"Methapharm, Inc.",15.0,0.0,0.0
4,Oct 6 2022 11:49AM,248057,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2480332,69.0,28.0,3.0
12,248060,3.0,0.0,0.0
15,743959,27.0,45.0,0.0
16,496113,2.0,0.0,0.0
19,992196,4.0,0.0,0.0
21,247943,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2480332,69.0,28.0,3.0
1,12,248060,3.0,0.0,0.0
2,15,743959,27.0,45.0,0.0
3,16,496113,2.0,0.0,0.0
4,19,992196,4.0,0.0,0.0
5,21,247943,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,69.0,28.0,3.0
1,12,3.0,0.0,0.0
2,15,27.0,45.0,0.0
3,16,2.0,0.0,0.0
4,19,4.0,0.0,0.0
5,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,69.0
1,12,Released,3.0
2,15,Released,27.0
3,16,Released,2.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0
Executing,28.0,0.0,45.0,0.0,0.0,1.0
Released,69.0,3.0,27.0,2.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,0.0,45.0,0.0,0.0,1.0
2,Released,69.0,3.0,27.0,2.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,0.0,45.0,0.0,0.0,1.0
2,Released,69.0,3.0,27.0,2.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()